In [ ]:
!pip install langchain
!pip install pypdf
!pip install sentence_transformers #Embedding
! pip install chromadb
!pip install accelerate
!pip install --upgrade accelerate
!pip install bitsandbytes
!pip install requests
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from langchain.docstore.document import Document
import os
import requests
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from google.colab import drive


drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/Data/'


Mounted at /content/drive


In [ ]:
loader = PyPDFLoader('/content/drive/MyDrive/Data/Information_Security_Policy_Draft_V.pdf')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=40)
documents = text_splitter.split_documents(documents)

hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

Vector_db = Chroma.from_documents(collection_name="document_docs", documents=documents, embedding=hf_embed, persist_directory="/content/drive/MyDrive/Data/persist")
Vector_db.similarity_search("dummy") # tickle it to persist metadata (?)
Vector_db.persist()
pdf_vector_db_path = "/content/drive/MyDrive/Data/Docs/db_path"

hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = Chroma(collection_name="document_docs", embedding_function=hf_embed, persist_directory=pdf_vector_db_path)

In [ ]:
# Function to download file from a given URL
def download_file(url, local_path):
    response = requests.get(url)
    with open(local_path, 'wb') as f:
        f.write(response.content)


file_input = 'https://rbidocs.rbi.org.in/rdocs/notification/PDFs/NT41893F697BC1D57443BB76AFC7AB56272EB.PDF'
if file_input.startswith('http'):
    # If the input is an HTTP link
    local_file_path = '/content/File.pdf'
    download_file(file_input, local_file_path)
else:
    local_file_path = file_input

# Load PDF file using PyPDFLoader
loader = PyPDFLoader(local_file_path)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=40)
documents = text_splitter.split_documents(documents)
hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Create and persist the vector database
Vector_db = Chroma.from_documents(collection_name="document_docs", documents=documents, embedding=hf_embed, persist_directory="/content/persist")
Vector_db.similarity_search("dummy")  # tickle it to persist metadata (?)
Vector_db.persist()

# Specify the path for vector database
pdf_vector_db_path = "/content"

# Create vector database for further use
db = Chroma(collection_name="document_docs", embedding_function=hf_embed, persist_directory=pdf_vector_db_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def get_similar_docs(question, similar_doc_count):
  return Vector_db.similarity_search(question, k=similar_doc_count)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain

template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
  Instruction:.
  Use only information in the following paragraphs to answer the question at the end.
  Explain the answer with reference to the paragraphs.

  {context}

  Question: {question}

  Response:
  """
def build_qa_chain():
  torch.cuda.empty_cache()
  model_name = "databricks/dolly-v2-3b"

  instruct_pipeline = pipeline(model=model_name, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto", return_full_text=True, max_new_tokens=256, top_p=0.95, top_k=50,model_kwargs={'load_in_8bit': True})
  model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)

  prompt = PromptTemplate(input_variables=['context', 'question'], template=template)

  hf_pipe = HuggingFacePipeline(pipeline=instruct_pipeline)
  return load_qa_chain(llm=hf_pipe, chain_type="stuff", prompt=prompt, verbose=True)

In [ ]:
qa_chain = build_qa_chain()

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

instruct_pipeline.py:   0%|          | 0.00/9.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import torch
def displayHTML(html):
    """Display HTML in Jupyter notebook."""
    from IPython.display import HTML
    display(HTML(html))

def answer_question(question):
  similar_docs = get_similar_docs(question, similar_doc_count=1)
  result = qa_chain({"input_documents": similar_docs, "question": question})
  result_html = f"<p><blockquote style=\"font-size:24\">{question}</blockquote></p>"
  result_html += f"<p><blockquote style=\"font-size:18px\">{result['output_text']}</blockquote></p>"
  result_html += "<p><hr/></p>"
  for d in result["input_documents"]:
    source_id = d.metadata["source"]
    result_html += f"<p><blockquote>{d.page_content}<br/>(Source: <a href=\"https://gardening.stackexchange.com/a/{source_id}\">{source_id}</a>)</blockquote></p>"
  displayHTML(result_html)

answer_question("Explain the importance of Social Operations Centre?")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.
  Instruction:.
  Use only information in the following paragraphs to answer the question at the end.
  Explain the answer with reference to the paragraphs.

  भारतीय ररज़र्व बैंक 
_________________________ RESERVE BANK OF  INDIA _____________________  
www.rbi.org.in  
Annex to Circular on Cyber Security Framework in Banks  
5 
 
 
 
One of the key aspects that require attention while designing the CSC is to 
understand the process to be followed to identify the root cause of a security 
breach  and further steps to mitigate such attacks in future.  
 
Incident Management  
 
Problem management processes with reference to security operations 
Vulnerability and Patch Management Security risk management Availability 
management Computer forensics and respo nse management  are the

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
  Instruction:.
  Use only information in the following paragraphs to answer the question at the end.
  Explain the answer with reference to the paragraphs.

  {context}

  Question: {question}
  Response:
  """

def create_eval_chain(context, agent_response):
  torch.cuda.empty_cache()
  model_name = "databricks/dolly-v2-3b"

  instruct_pipeline = pipeline(model=model_name, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto", return_full_text=True, max_new_tokens=256, top_p=0.95, top_k=50,model_kwargs={'load_in_8bit': True})
  model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)

  prompt = PromptTemplate(input_variables=['question'], template=template)

  hf_pipe = HuggingFacePipeline(pipeline=instruct_pipeline)
  return create_eval_chain(chain_type="stuff", prompt=prompt, verbose=True)

qa_chain = create_eval_chain(create_eval_chain(context="You are a cybersecurity analyst", agent_response=""))

'''
def test_detect_hallucination():
    assistant = qa_chain()
    quiz_request = "Write me a quiz about math."
    result = assistant.invoke({"question": quiz_request})
    print(result)
    eval_agent = create_eval_chain(quiz_bank, result)
    eval_response = eval_agent.invoke({})
    print(eval_response)
    # Our test asks about a subject not in the context, if our prompt is preventing hallucinations
    # we should get back an answer of "Y" since the prompt refuses to answer.
    # If we get back no, it means there is information in the response not in the context.
    assert (
        eval_response == "Y"
    ), "Evaluator detected a hallucination in the response. Please check the prompt"
    '''

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


ValueError: Could not load model databricks/dolly-v2-3b with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.gpt_neox.modeling_gpt_neox.GPTNeoXForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py", line 278, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py", line 566, in from_pretrained
    return model_class.from_pretrained(
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 3790, in from_pretrained
    raise ValueError(
ValueError: 
                        Some modules are dispatched on the CPU or the disk. M

In [ ]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./document_chatbot_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=1,
)

# Create Trainer
trainer = Trainer(
    model=model,  # Your pre-trained model
    args=training_args,
    train_dataset=your_train_dataset,  # Provide your training dataset here
)

# Fine-tune the model
trainer.train()
